In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Visualization of the second Riemann sheet with rotated branch cut 
TR-979
^^^
To reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=4) of the PDG way to get from the first physical sheet to the second unphysical sheet is derived for the Amplitude calculated within the K matrix formalism.   
:::
::::

# Riemann sheets for one channel 

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.17.0 sympy==1.12

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math
from ipywidgets import FloatSlider, VBox, interactive_output

warnings.filterwarnings("ignore")

## Rotated square root definition 
See report on Riemann sheets for one channel case [Link]

In [ ]:
@unevaluated
class RotatedSqrt(sp.Expr):
    z: Any
    phi: Any
    _latex_repr_ = R"\sqrt[{phi}]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z, phi = self.args
        return sp.exp(-phi * sp.I / 2) * sp.sqrt(z * sp.exp(phi * sp.I))


z, phi = sp.symbols("z phi")
expr = RotatedSqrt(z, phi)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor_Rotated(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    phi: Any
    _latex_repr_ = R"\rho^\phi_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2, phi = self.args
        return RotatedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2, phi)


s, m1, m2, phi = sp.symbols("s m1 m2 phi")
rho_expr_rotated = PhaseSpaceFactor_Rotated(s, m1, m2, phi)
Math(aslatex({rho_expr_rotated: rho_expr_rotated.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

In [ ]:
from ampform.kinematics.phasespace import Kallen


@unevaluated(real=False)
class PhaseSpaceFactorKallen(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return 2 * BreakupMomentum(s, m1, m2) / sp.sqrt(s)


@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr_kallen = PhaseSpaceFactorKallen(s, m1, m2)
rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
kallen = Kallen(*sp.symbols("x:z"))
Math(
    aslatex({
        e: e.doit(deep=False)
        for e in [rho_expr_kallen, rho_cm_expr, cm_expr, q_expr, kallen]
    })
)

:::{note} 
It seems like the real part of the Chew-Mandelstam function is switching signs when coming from the $-i\epsilon$ direction below the branch cut above $s_{thr}$.
Therefore we set:
$$
-i\cdot\rho_{-\epsilon} = CM_{-\epsilon}
$$
this leads:
$$
\rho_{-\epsilon} = i\cdot CM_{-\epsilon}
$$
As the Phasespace from the Chew-Mandelstam function is defined as:
$$
\rho_{CM} = -16\cdot\pi\cdot i \cdot CM
$$
in the PhaseSpaceCM class one has to add a minus sign in the following calculations. 
:::

### Visual comparison

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget
symbols_cm = sp.Tuple(s, m1, m2)
symbols_rho = sp.Tuple(s, m1, m2, phi)
rho_func_rotated = sp.lambdify(symbols_rho, rho_expr_rotated.doit())
cm_func = sp.lambdify(symbols_cm, rho_cm_expr.doit())
rho_func = sp.lambdify(symbols_cm, rho_expr.doit())

ϵi = 1e-7 * 1j
fig, axes = plt.subplots(
    figsize=(6, 8.5),
    gridspec_kw=dict(),
    nrows=2,
)
ax_1, ax_2 = axes
ax_2.set_ylabel(R"$\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)$")
ax_1.set_ylabel(R"$\rho_{{{m1},{m2}}}\left({s}\right)$")
for ax in axes:
    ax.set_xlabel("$\mathrm{Re}\,s$")

data = None
x = np.linspace(-4, 4, 400)
m1_val = 0.5
m2_val = 0.6
args_1 = (m1_val, m2_val)
sliders = dict(
    phi=FloatSlider(
        min=-3 * np.pi,
        max=+3 * np.pi,
        step=np.pi / 8,
        description="phi",
    ),
)


def plot(phi):
    global data
    ax_1.set_title(R"Phasespace factor")
    ax_2.set_title(R"Chew-Mandelstam")
    cm = cm_func(x + ϵi, *args_1)
    rho_rotated = rho_func_rotated(x, *args_1, phi)
    rho = rho_func(x + ϵi, *args_1)
    cm_n = cm_func(x - ϵi, *args_1)
    rho_n = rho_func(x - ϵi, *args_1)
    if data is None:
        data = {
            "real_rho": ax_1.plot(x, rho.real, label="real_rho", color="blue")[0],
            "imag_rho": ax_1.plot(x, rho.imag, label="imag_rho", color="green")[0],
            "real_rho_n": ax_1.plot(
                x, rho_n.real, label="real_rho_n", color="magenta", linestyle="--"
            )[0],
            "imag_rho_n": ax_1.plot(
                x, rho_n.imag, label="imag_rho_n", color="cyan", linestyle="--"
            )[0],
            "real_rho_rotated": ax_1.plot(
                x, rho.real, label="real_rho_rotated", color="blue", linestyle="--"
            )[0],
            "imag_rho_rotated": ax_1.plot(
                x, rho.imag, label="imag_rho_rotated", color="green", linestyle="--"
            )[0],
            "real_CM": ax_2.plot(x, (cm).real, label="real_cm", color="red")[0],
            "imag_CM": ax_2.plot(x, (cm).imag, label="imag_cm", color="yellow")[0],
            "real_CM_n": ax_2.plot(
                x, -(cm_n).real, label="real_cm_n", color="red", linestyle="--"
            )[0],
            "imag_CM_n": ax_2.plot(
                x, -(cm_n).imag, label="imag_cm_n", color="yellow", linestyle="--"
            )[0],
        }
    else:
        data["real_rho"].set_ydata(rho.real)
        data["imag_rho"].set_ydata(rho.imag)
        data["real_rho_n"].set_ydata(rho_n.real)
        data["imag_rho_n"].set_ydata(rho_n.imag)
        data["real_rho_rotated"].set_ydata(rho_rotated.real)
        data["imag_rho_rotated"].set_ydata(rho_rotated.imag)
        data["real_CM"].set_ydata(cm.real)
        data["imag_CM"].set_ydata(cm.imag)
        data["real_CM_n"].set_ydata(-cm_n.real)
        data["imag_CM_n"].set_ydata(-cm_n.imag)
    fig.canvas.draw_idle()


ui = VBox(tuple(sliders.values()))
output = interactive_output(plot, controls=sliders)
ax_1.legend(loc="lower left")
ax_2.legend(loc="lower left")
ax_1.set_ylim(-1, 1)
ax_2.set_ylim(-1, 1)
fig.tight_layout()
display(ui, output)

## T matrix definition with K matrix - 1 channel case 
In the following the dynamical part of the Amplitude is described within the K matrix formalism. Due to encircling the the right hand branch cut of the phasespace factor by using the Chew-Mandelstam function the Amplitude function of teh first sheet is described. The function is defined for positive real values and positive and negative imaginary values. 

In [ ]:
n_channels = 1
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)

CM_symbol = sp.MatrixSymbol(R"{\rho_{\Sigma}}", n_channels, n_channels)
CM = sp.DiagonalMatrix(CM_symbol)
CM.as_explicit()

In [ ]:
T_I = (I - sp.I * K * CM).inv() * K
T_I

In [ ]:
T_I_explicit = T_I.as_explicit()
T_I_explicit

:::{note}
No Blatt-Weisskopf form factors because $L=0$.
:::

In [ ]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
w0 = sp.Symbol("Gamma0")
gamma1 = sp.Symbol("gamma1")
gamma2 = sp.Symbol("gamma2")
phi_rot = sp.Symbol("phi")
symbols = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2)
symbols_rot = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2, phi_rot)
k_expr_00 = (gamma1 * gamma1 * m0 * w0) / (s - m0**2)
k_expr_10 = (gamma1 * gamma2 * m0 * w0) / (s - m0**2)
k_expr_11 = (gamma2 * gamma2 * m0 * w0) / (s - m0**2)

In [ ]:
cm_expressions = {
    K[0, 0]: k_expr_00,
    # K[1, 1]: k_expr_11,
    # K[0, 1]: k_expr_10,
    # K[1, 0]: k_expr_10,
    CM[0, 0]: -PhaseSpaceCM(s, ma1, mb1),
    # CM[1, 1]: PhaseSpaceCM(s, ma2, mb2),
}
(
    (1j * PhaseSpaceCM(0.7**2 + ϵi, 0.2, 0.4).doit()).n(),
    (1j * PhaseSpaceCM(0.7**2 - ϵi, 0.2, 0.4).doit()).n(),
)

In [ ]:
T_I_cm_expr_00 = T_I_explicit[0, 0].xreplace(cm_expressions)

In [ ]:
T_I_cm_expr_00.simplify(doit=False)

## Second Riemann sheet 
Since the $T$ function is real below the branch cut it can be shown that the discontinuity above and below the threshold reads as:
$$
CM(s+i\epsilon)-CM(s-i\epsilon)= i\rho -(-i\rho) =2i\rho
$$
when $\epsilon$ goes to zero.<br>
Which leads:
$$
CM^{-1}_{II}(s-i\epsilon)= Re(CM^{-1}_{I}(s-i\epsilon))-i\rho+2i\rho
$$
For the the Amplitude for the second sheet is defined as:
$$
A^{-1}_{II}(s)= A^{-1}_{I}(s)-2i\rho
$$
The Amplitude for the second sheet is only defined for $s$ positive real part and negative complex. It inherits the analytic structure of the phasespace factor $\rho$ (the branch cut starting form zero and from $s=s_{thr}$ on the real axis). So it is only defined up to the closest branch cut which is in this case the cut at $s=s_{thr}$. This cut can be rotated in an arbitrary direction pointing to the right or downwards.  

In [ ]:
rho_symbol = sp.MatrixSymbol("rho", n_channels, n_channels)
rho = sp.DiagonalMatrix(rho_symbol)
rho.as_explicit()

In [ ]:
T_II = (T_I.inv() + 2 * sp.I * rho).inv()

In [ ]:
T_II_explicit = T_II.as_explicit()
T_II_explicit

In [ ]:
rho_expressions = {
    **cm_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    # rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}

In [ ]:
rho_expressions_rotated = {
    **cm_expressions,
    rho[0, 0]: PhaseSpaceFactor_Rotated(s, ma1, mb1, phi),
    # rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}

In [ ]:
T_II_rho_expr_00 = T_II_explicit[0, 0].xreplace(rho_expressions)
T_II_rho_expr_rotated_00 = T_II_explicit[0, 0].xreplace(rho_expressions_rotated)

In [ ]:
T_II_rho_expr_00.simplify(doit=False)

In [ ]:
T_II_rho_expr_rotated_00.simplify(doit=False)

In [ ]:
T_I_func = sp.lambdify(symbols, T_I_cm_expr_00.doit())
T_II_func = sp.lambdify(symbols, T_II_rho_expr_00.doit())
values = {
    ma1: 0.9,
    mb1: 0.8,
    ma2: 10,
    mb2: 10,
    m0: 3.1,
    w0: 1.0,
    gamma1: 1,
    gamma2: 1,
}

args = eval(str(symbols[1:].xreplace(values)))

epsilon = 1e-5
x = np.linspace(0, 6, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

T1n = T_I_func(Zn**2, *args)
T1p = T_I_func(Zp**2, *args)

T2n = T_II_func(Zn**2, *args)
T2p = T_II_func(Zp**2, *args)

fig, axes = plt.subplots(figsize=(10, 6), ncols=2, sharey=True)
ax1, ax2 = axes
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(s)$")

ax1.plot(x, T1n[0].imag, label=R"$T_\mathrm{I}(s-0i)$")
ax1.plot(x, T1p[0].imag, label=R"$T_\mathrm{I}(s+0i)$")
ax1.set_title(f"${sp.latex(rho_cm_expr)}$")
ax1.set_title(R"$T_\mathrm{I}$")

ax2.plot(x, T2n[0].imag, label=R"$T_\mathrm{II}(s-0i)$")
ax2.plot(x, T2p[0].imag, label=R"$T_\mathrm{II}(s+0i)$")
ax2.set_title(R"$T_\mathrm{II}$")

for ax in axes:
    ax.legend()

fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ["png"]
fig, axes_2 = plt.subplots(figsize=(10, 6), ncols=3, sharey=True)
ax3, ax4, ax5 = axes_2
ax3.set_xlabel(R"$\mathrm{Re}\,s$")
ax3.set_ylabel(R"$\mathrm{Im}\,s$")
T_max = 1.5
mesh1 = ax3.pcolormesh(X, Y, T1p.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh2 = ax4.pcolormesh(X, -Y, T2n.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh3 = ax5.pcolormesh(X, Y, T1p.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh3 = ax5.pcolormesh(X, -Y, T2n.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
plt.colorbar(mesh1, ax=ax5, pad=0.02)
fig.tight_layout()
plt.show()

## Investigation on rotation the branch cut of the second sheet 

In [ ]:
T_II_func_rot = sp.lambdify(symbols_rot, T_II_rho_expr_rotated_00.doit())
values_rot = {
    ma1: 0.9,
    mb1: 0.8,
    ma2: 10,
    mb2: 10,
    m0: 3.1,
    w0: 1.0,
    gamma1: 1,
    gamma2: 1,
    phi_rot: 2 * np.pi,
}
args_rot = eval(str(symbols_rot[1:].xreplace(values_rot)))

T2n_rot = T_II_func_rot(Zn**2, *args_rot)
T2p_rot = T_II_func_rot(Zp**2, *args_rot)
s_thr = values_rot[ma1] + values_rot[mb1]

In [ ]:
%config InlineBackend.figure_formats = ["png"]
fig_3, axes_3 = plt.subplots(figsize=(10, 6), ncols=3, sharey=True)
ax6, ax7, ax8 = axes_3
ax6.set_xlabel(R"$\mathrm{Re}\,s$")
ax6.set_ylabel(R"$\mathrm{Im}\,s$")
T_max = 1.5
mesh4 = ax6.pcolormesh(X, Y, T1p.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh5 = ax6.pcolormesh(X, -Y, T1n.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh6 = ax7.pcolormesh(
    X, Y, T2p_rot.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm
)
mesh7 = ax7.pcolormesh(
    X, -Y, T2n_rot.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm
)
mesh8 = ax8.pcolormesh(X, Y, T1p.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh9 = ax8.pcolormesh(
    X, -Y, T2n_rot.imag, vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm
)


plt.colorbar(mesh4, ax=ax8, pad=0.02)
fig_3.tight_layout()
plt.show()

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    n_lines = 16
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        contours=dict(
            x=dict(
                show=True,
                start=x.min(),
                end=x.max(),
                size=(x.max() - x.min()) / n_lines,
                color="black",
            ),
            y=dict(
                show=True,
                start=-y.max(),
                end=+y.max(),
                size=(y.max() - y.min()) / (n_lines // 2),
                color="black",
            ),
        ),
        name=sheet_name,
        opacity=0.6,
        showscale=False,
    )


vmax = 1.6
project = np.imag
sheet_colors = {
    "Physical (T1)": "blue",
    "Unphysical (T2)": "red",
}

In [ ]:
Sn = go.Surface(x=X, y=-Y, z=T2n.imag, **sty("Unphysical (T2)"))
Sp = go.Surface(x=X, y=+Y, z=T1p.imag, **sty("Physical (T1)"))
zeros = np.zeros(x.shape)
lineshape = go.Scatter3d(
    x=x,
    y=zeros,
    z=project(T1p[0]),
    line=dict(color="yellow", width=10),
    mode="lines",
    name="Lineshape",
)

fig = go.Figure(data=[Sn, Sp, lineshape])
fig.update_layout(
    height=550,
    margin=dict(l=0, r=0, t=30, b=0),
    showlegend=False,
    title_text="Im(T) with Chew-Mandelstam phase space factor",
)
fig.update_scenes(
    camera_center=dict(z=-0.2),
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_range=[-vmax, +vmax],
)

fig.show()